<a href="https://colab.research.google.com/github/qamtam/Hands-on-machine-learning/blob/main/CH15_SET10_with_notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# Common imports
import numpy as np
import os
from pathlib import Path
np.set_printoptions(threshold=np.inf) # print pretty pony tables
np.set_printoptions(linewidth=2000)
# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "rnn"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [ ]:
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/datasets/jsb_chorales/jsb_chorales.tgz"
FILENAME = "jsb_chorales.tgz"

filepath = keras.utils.get_file(FILENAME,
                                DOWNLOAD_ROOT,
                                cache_subdir="datasets/bach",
                                extract=True)

122880/117661 [===============================] - 0s 0us/step


In [ ]:
root = os.path.join("/root/.keras/datasets/bach")
bach_dir = Path(filepath).parent
os.path.abspath(bach_dir)
os.listdir(bach_dir)

['train', 'valid', 'jsb_chorales.tgz', 'test']

NameError: ignored

In [ ]:
train_path = bach_dir / "train"
val_path = bach_dir / "valid"
test_path = bach_dir / "test"

In [ ]:
train_files = [str(path) for path in os.listdir(train_path)]
valid_files = [str(path) for path in os.listdir(val_path)]
test_files = [str(path) for path in os.listdir(test_path)]


In [ ]:
train_paths = [os.path.join(os.path.abspath(bach_dir), "train", x) for x in train_files]
valid_paths = [os.path.join(os.path.abspath(bach_dir), "valid", x) for x in valid_files]
test_paths = [os.path.join(os.path.abspath(bach_dir), "test", x) for x in test_files]

n_inputs=4


def preprocess(line):
  defs = [0.] * n_inputs
  fields = tf.io.decode_csv(line, record_defaults=defs)
  results = tf.stack(fields)
  return results

def read_bach_notes_from_csv(filepaths, repeat=1, n_readers=5, n_read_threads=None, shuffle_buffer_size=10000, n_parse_threads=5, batch_size=32):
  dataset= tf.data.Dataset.list_files(filepaths) #creates a dataset of filepaths
  dataset = dataset.map(lambda x: tf.data.TextLineDataset(x))
  for x in dataset:
    print(x)
  #dataset = dataset.map(preprocess)
  dataset = dataset.repeat(repeat)
  return dataset.batch(batch_size).prefetch(1)

In [ ]:



train = read_bach_notes_from_csv(train_paths)
valid = read_bach_notes_from_csv(valid_paths)
test = read_bach_notes_from_csv(test_paths)

In [ ]:
for x in train.take(100):
  print(x)

tf.Tensor(
[[74. 69. 66. 62.]
 [74. 69. 66. 62.]
 [74. 69. 64. 61.]
 [74. 69. 64. 61.]
 [74. 69. 62. 59.]
 [74. 69. 62. 59.]
 [74. 67. 62. 59.]
 [74. 67. 62. 59.]
 [74. 66. 60. 57.]
 [74. 66. 60. 57.]
 [74. 67. 59. 55.]
 [74. 67. 59. 55.]
 [69. 69. 62. 54.]
 [69. 69. 62. 54.]
 [71. 66. 62. 54.]
 [71. 66. 62. 54.]
 [73. 67. 64. 52.]
 [73. 67. 64. 52.]
 [73. 69. 64. 52.]
 [73. 69. 64. 52.]
 [74. 66. 57. 50.]
 [74. 66. 57. 50.]
 [73. 64. 57. 50.]
 [73. 64. 57. 50.]
 [71. 62. 57. 52.]
 [71. 62. 57. 52.]
 [71. 64. 56. 52.]
 [71. 64. 56. 52.]
 [69. 61. 57. 45.]
 [69. 61. 57. 45.]
 [69. 61. 57. 45.]
 [69. 61. 57. 45.]], shape=(32, 4), dtype=float32)
tf.Tensor(
[[74. 66. 54. 47.]
 [74. 66. 54. 47.]
 [74. 66. 56. 47.]
 [74. 66. 56. 47.]
 [73. 64. 57. 49.]
 [73. 64. 57. 49.]
 [73. 66. 57. 50.]
 [73. 66. 57. 50.]
 [71. 67. 64. 52.]
 [71. 67. 64. 52.]
 [71. 67. 62. 52.]
 [71. 67. 62. 52.]
 [69. 66. 61. 54.]
 [69. 66. 61. 54.]
 [69. 64. 61. 54.]
 [69. 64. 61. 54.]
 [71. 62. 59. 55.]
 [71. 62. 59. 5

InvalidArgumentError: ignored

In [ ]:
zet = train_paths[0]
zet

'/root/.keras/datasets/bach/train/chorale_127.csv'

In [ ]:
data = tf.data.TextLineDataset(zet)
for x in data:
  print(x)

In [ ]:
z = train.take(1)

In [ ]:
files = tf.data.Dataset.list_files(train_paths)
for x in files:

  print(x)
  print("new data set instance in the dataset \n")
  
dataset = files.map(lambda x: tf.string.to_string(x))
""""

dataset = files.map(lambda yomamma: tf.data.TextLineDataset(yomamma.as_string).skip(1))
for x in dataset:
  print(x)
"""

tf.Tensor(b'/root/.keras/datasets/bach/train/chorale_041.csv', shape=(), dtype=string)
new data set instance in the dataset 

tf.Tensor(b'/root/.keras/datasets/bach/train/chorale_033.csv', shape=(), dtype=string)
new data set instance in the dataset 

tf.Tensor(b'/root/.keras/datasets/bach/train/chorale_161.csv', shape=(), dtype=string)
new data set instance in the dataset 

tf.Tensor(b'/root/.keras/datasets/bach/train/chorale_109.csv', shape=(), dtype=string)
new data set instance in the dataset 

tf.Tensor(b'/root/.keras/datasets/bach/train/chorale_163.csv', shape=(), dtype=string)
new data set instance in the dataset 

tf.Tensor(b'/root/.keras/datasets/bach/train/chorale_062.csv', shape=(), dtype=string)
new data set instance in the dataset 

tf.Tensor(b'/root/.keras/datasets/bach/train/chorale_185.csv', shape=(), dtype=string)
new data set instance in the dataset 

tf.Tensor(b'/root/.keras/datasets/bach/train/chorale_171.csv', shape=(), dtype=string)
new data set instance in the da

AttributeError: ignored

In [ ]:
z = 0
for item in valid.take(1):
  print(item)
print(z)

In [ ]:
from IPython.display import Audio

def notes_to_frequencies(notes):
    # Frequency doubles when you go up one octave; there are 12 semi-tones
    # per octave; Note A on octave 4 is 440 Hz, and it is note number 69.
    return 2 ** ((np.array(notes) - 69) / 12) * 440

def frequencies_to_samples(frequencies, tempo, sample_rate):
    note_duration = 60 / tempo # the tempo is measured in beats per minutes
    # To reduce click sound at every beat, we round the frequencies to try to
    # get the samples close to zero at the end of each note.
    frequencies = np.round(note_duration * frequencies) / note_duration
    n_samples = int(note_duration * sample_rate)
    time = np.linspace(0, note_duration, n_samples)
    sine_waves = np.sin(2 * np.pi * frequencies.reshape(-1, 1) * time)
    # Removing all notes with frequencies ≤ 9 Hz (includes note 0 = silence)
    sine_waves *= (frequencies > 9.).reshape(-1, 1)
    return sine_waves.reshape(-1)

def chords_to_samples(chords, tempo, sample_rate):
    freqs = notes_to_frequencies(chords)
    freqs = np.r_[freqs, freqs[-1:]] # make last note a bit longer
    merged = np.mean([frequencies_to_samples(melody, tempo, sample_rate)
                     for melody in freqs.T], axis=0)
    n_fade_out_samples = sample_rate * 60 // tempo # fade out last note
    fade_out = np.linspace(1., 0., n_fade_out_samples)**2
    merged[-n_fade_out_samples:] *= fade_out
    return merged

def play_chords(chords, tempo=160, amplitude=0.1, sample_rate=44100, filepath=None):
    samples = amplitude * chords_to_samples(chords, tempo, sample_rate)
    if filepath:
        from scipy.io import wavfile
        samples = (2**15 * samples).astype(np.int16)
        wavfile.write(filepath, sample_rate, samples)
        return display(Audio(filepath))
    else:
        return display(Audio(samples, rate=sample_rate))




In [ ]:
import tensorflow_datasets as tfds
my_chords = tfds.as_numpy(
    train.take(1)
)

In [ ]:
my_chords

In [ ]:
import pandas as pd
import numpy as np
def f(l):
    if isinstance(l,(list,pd.core.series.Series,np.ndarray)):
        print(5)
    else:
        raise Exception('wrong type')


chords = np.array([[0,0,0,0]])
for ex in my_chords:
  chords = np.append(chords, ex, axis=0)

In [ ]:
chords[0]

In [ ]:
for i in range(chords.shape[0]):
  play_chords(chords)